In [1]:
import sys
import cv2
sys.path.append('../disk')

In [2]:
from disk import DISK
import detect

2025-01-14 21:09:15.162218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736885355.185256  177815 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736885355.191101  177815 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 21:09:15.212535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
disk = DISK()

In [4]:
dataset = detect.SceneDataset('../data/segmented_dataset/0/', crop_size = (640, 640))

In [5]:
import torch
state_dict = torch.load('../disk/depth-save.pth', map_location='cpu')

/tmp/ipykernel_177815/2407649423.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('../disk/depth-save.pth', map_location='cpu')


In [6]:
weights = state_dict['extractor']

In [7]:
model = DISK(window = 8, desc_dim=128)

In [8]:
model.load_state_dict(weights)

<All keys matched successfully>

In [9]:
model = model.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

In [10]:
described_samples = detect.extract(dataset, '.', model)

100%|██████████| 12/12 [00:22<00:00,  1.91s/it, n=2161]


In [11]:
import pandas as pd

In [12]:
pd.read_csv('../data/processed_metadata.csv').head()

,image_id,identity,path,bbox,keypoints,original_split,processed_path
0,0004d0b59e19461f,34,atrw_reid_train/train/001970.jpg,NaN,[326 77 2 283 86 2 331 141 2 210 174 ...,train,./data/segmented_dataset/34
1,000871c1fc726f0b,177,atrw_reid_train/train/004612.jpg,NaN,[127 57 2 144 82 2 93 149 2 264 183 ...,train,./data/segmented_dataset/177
2,00411460f7c92d21,112,atrw_reid_test/test/000363.jpg,NaN,[ 0 0 0 537 28 2 562 77 2 437 135 ...,test,./data/segmented_dataset/112
3,0060ef47b12160b9,18,atrw_reid_test/test/003492.jpg,NaN,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,test,./data/segmented_dataset/18
4,006f52e9102a8d3b,19,atrw_reid_test/test/000168.jpg,NaN,[1345 170 2 1520 238 2 1471 400 2 ...,test,./data/segmented_dataset/19
